### Importing Packages

In [15]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableLambda, RunnableBranch
from dotenv import load_dotenv

load_dotenv()

True

### Model Setup

In [2]:
model = ChatGroq(
    model="meta-llama/llama-4-maverick-17b-128e-instruct",
    temperature=0.7,
    max_tokens=1024,
) 

### Runnable Sequence

In [5]:
prompt1 = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Explain the following joke - {text}',
    input_variables=['text']
)

parser = StrOutputParser()

In [7]:
chain = RunnableSequence(prompt1, model, parser, prompt2, model, parser)
print(chain.invoke({'topic':'AI'}))

The joke is a play on words. "Process" has a double meaning here:

1. In computing, "process" refers to the act of handling or executing a task, such as data processing. AI systems are designed to process information.
2. In a psychological context, "processing emotions" means to understand, deal with, and manage one's feelings.

The joke is funny because it takes the literal meaning of "process" (related to computing) and applies it to the AI's emotional state, creating a pun. The AI is "struggling to process its emotions" in the sense that it's having trouble understanding and managing its feelings, just like a human might. The wordplay creates a humorous connection between the AI's computational nature and its supposed emotional struggles.


### Runnable Parallel

In [9]:
prompt1 = PromptTemplate(
    template='Generate a tweet about {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Generate a Linkedin post about {topic}',
    input_variables=['topic']
)

parser = StrOutputParser()

parallel_chain = RunnableParallel({
    'tweet': RunnableSequence(prompt1, model, parser),
    'linkedin': RunnableSequence(prompt2, model, parser)
})

In [11]:
result = parallel_chain.invoke({'topic':'AI'})
print("Twitter\n",result['tweet'])
print("LinkedIn\n",result['linkedin'])

Twitter
 "Just learned about the latest advancements in #AI and I'm blown away! From revolutionizing healthcare to transforming transportation, the possibilities are endless! What do you think is the most exciting application of AI right now? #ArtificialIntelligence #Tech"
LinkedIn
 Here's a sample LinkedIn post about AI:

**Title:** "The Future is Here: How AI is Revolutionizing Industries"

**Post:**

"Artificial Intelligence (AI) is no longer just a buzzword - it's a reality that's transforming the way we work, live, and interact with each other. From automating mundane tasks to enabling groundbreaking innovations, AI is revolutionizing industries across the globe.

In recent years, we've seen significant advancements in AI technology, with applications in areas such as:

* **Predictive Maintenance**: AI-powered predictive maintenance is helping companies reduce downtime and increase efficiency in industries like manufacturing and energy.
* **Personalized Medicine**: AI-driven medic

### Runnable Passthrough

In [22]:
prompt1 = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Explain the following joke - {text}',
    input_variables=['text']
)

parser = StrOutputParser()

joke_gen_chain = RunnableSequence(prompt1, model, parser)

parallel_chain = RunnableParallel({
    'joke': RunnablePassthrough(),
    'explanation': RunnableSequence(prompt2, model, parser)
})

In [23]:
final_chain = RunnableSequence(joke_gen_chain, parallel_chain)
# final_chain = joke_gen_chain | parallel_chain

print(final_chain.invoke({'topic':'cricket'}))

{'joke': 'Why did the cricket go to the doctor?\n\nBecause it had a "pitch" black bruise! (get it? pitch, like in cricket!)', 'explanation': 'The joke is a play on words. "Pitch" has a double meaning here:\n\n1. In cricket, a pitch refers to the area of the ground where the game is played. It\'s a term specific to the sport.\n2. "Pitch black" is an idiomatic expression meaning extremely dark or black.\n\nThe joke is saying that the cricket (the insect, not the sport) went to the doctor because it had a bruise that was "pitch black", meaning it was very dark or bruised. The punchline is funny because it\'s making a connection between the cricket (the insect) and the sport of cricket, using the word "pitch" in a clever and unexpected way.\n\nThe humor comes from the unexpected twist on the word "pitch" and the clever connection made between the setup (the cricket going to the doctor) and the punchline (the "pitch" black bruise). The joke relies on a basic understanding of the sport of cr

### Runnable Lambda

In [16]:
def word_count(text):
    return len(text.split())

prompt = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

parser = StrOutputParser()

joke_gen_chain = RunnableSequence(prompt, model, parser)

parallel_chain = RunnableParallel({
    'joke': RunnablePassthrough(),
    'word_count': RunnableLambda(word_count)
})

In [17]:
final_chain = RunnableSequence(joke_gen_chain, parallel_chain)
result = final_chain.invoke({'topic':'AI'})
final_result = """{} \n word count - {}""".format(result['joke'], result['word_count'])
print(final_result)

Why did the AI go on a diet?

Because it wanted to lose some bytes! 
 word count - 15


### Runnable Branch

In [1]:
prompt1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Summarize the following text \n {text}',
    input_variables=['text']
)

parser = StrOutputParser()

report_gen_chain = prompt1 | model | parser

branch_chain = RunnableBranch(
    (lambda x: len(x.split())>30, prompt2 | model | parser),
    RunnablePassthrough()
)

NameError: name 'PromptTemplate' is not defined

In [19]:
final_chain = RunnableSequence(report_gen_chain, branch_chain)
print(final_chain.invoke({'topic':'Russia vs Ukraine'}))

### Summary of the Russia-Ukraine Conflict Report

The Russia-Ukraine conflict is a complex and ongoing crisis that began in 2014. The conflict has its roots in Ukraine's historical ties with Russia, its aspirations to integrate with the European Union, and Russia's strategic interests in the region.

#### Key Causes and Events

The conflict is driven by several factors, including:

* **Ukraine's pro-European aspirations**: Ukraine's desire to integrate with the EU and adopt Western values is seen as a threat to Russia's regional dominance.
* **Russia's strategic interests**: Russia's interests in Ukraine are driven by its need to maintain access to Ukraine's ports, particularly in Crimea, and to prevent Ukraine's integration with NATO.
* **Ethnic and linguistic divisions**: Ukraine's population is divided between Ukrainian-speaking nationalists and Russian-speaking populations, particularly in the eastern regions of Donetsk and Luhansk.

Key events that have shaped the conflict includ